# Video Frame Extraction

The cell below imports necessary modules, sets up the input and output directories, and calls the `process_all_videos` function to extract frames from all video files in the input directory and save them in the output directory.

In [1]:
import sys

# Add the path to the scripts directory
sys.path.append('../scripts')
# Import the process_all_videos function from frame_extractor.py
from frame_extractor import process_all_videos

# Define the input and output directories
input_dir = "../data/videos"
output_dir = "../output/video_frames/"

# Call the function to process all videos
# This will extract frames from all video files in the input directory and its subdirectories
# and save the frames in the output directory
process_all_videos(input_dir, output_dir)

Extracting frames from videos in c:\Users\yoopo\Documents\samplingframework\data\videos to c:\Users\yoopo\Documents\samplingframework\output\video_frames...
Processing folder: c:\Users\yoopo\Documents\samplingframework\data\videos
Processing folder: c:\Users\yoopo\Documents\samplingframework\data\videos\408
Found video: c:\Users\yoopo\Documents\samplingframework\data\videos\408\VID002A.mp4
c:\Users\yoopo\Documents\samplingframework\output\video_frames\408\VID002A
Extracted 5002 frames from c:\Users\yoopo\Documents\samplingframework\data\videos\408\VID002A.mp4 at 2 fps.
Found video: c:\Users\yoopo\Documents\samplingframework\data\videos\408\VID002B.mp4
c:\Users\yoopo\Documents\samplingframework\output\video_frames\408\VID002B
Extracted 5002 frames from c:\Users\yoopo\Documents\samplingframework\data\videos\408\VID002B.mp4 at 2 fps.
Found video: c:\Users\yoopo\Documents\samplingframework\data\videos\408\VID002C.mp4
c:\Users\yoopo\Documents\samplingframework\output\video_frames\408\VID002

## Create COCO file
Below cells create coco files from extracted /output/video_frames and /data/ground_truth data

In [2]:
import os
import json
import numpy as np

categories =  [
    {"id": 1, "name": "Cerebellum", "supercategory": "shape"},
    {"id": 2, "name": "Arachnoid", "supercategory": "shape"},
    {"id": 3, "name": "CN8", "supercategory": "shape"},
    {"id": 4, "name": "CN5", "supercategory": "shape"},
    {"id": 5, "name": "CN7", "supercategory": "shape"},
    {"id": 6, "name": "CN_9_10_11", "supercategory": "shape"},
    {"id": 7, "name": "SCA", "supercategory": "shape"},
    {"id": 8, "name": "AICA", "supercategory": "shape"},
    {"id": 9, "name": "SuperiorPetrosalVein", "supercategory": "shape"},
    {"id": 10, "name": "Labrynthine", "supercategory": "shape"},
    {"id": 11, "name": "Vein", "supercategory": "shape"},
    {"id": 12, "name": "Brainstem", "supercategory": "shape"},
    {"id": 1001, "name": "Suction", "supercategory": "shape"},
    {"id": 1002, "name": "Bovie", "supercategory": "shape"},
    {"id": 1003, "name": "Bipolar", "supercategory": "shape"},
    {"id": 1004, "name": "Forcep", "supercategory": "shape"},
    {"id": 1005, "name": "BluntProbe", "supercategory": "shape"},
    {"id": 1006, "name": "Drill", "supercategory": "shape"},
    {"id": 1007, "name": "Kerrison", "supercategory": "shape"},
    {"id": 1008, "name": "Cottonoid", "supercategory": "shape"},
    {"id": 1009, "name": "Scissors", "supercategory": "shape"},
    {"id": 1012, "name": "Unknown", "supercategory": "shape"},
    {"id": 1023, "name": "Dissector", "supercategory": ""},
    {"id": 1024, "name": "Teflon", "supercategory": ""}
]

def converter(image_path, annotation_path):

        coco_annotations = []
        coco_images = []
        c = 0

        for filename in os.listdir(annotation_path):

            framename = filename.split('.')[0]
            frame_num = framename.split('.')[0][-4:]

            json_file_path = os.path.join(annotation_path,framename + '.json')
            with open(json_file_path, 'r') as f:
                result = json.load(f)

            if 'labels' in result and 'bboxes' in result and 'scores' in result and 'masks' in result:
                labels = result['labels']
                bboxes = result['bboxes']
                scores = result['scores']

                # segm results
                masks = result['masks']
                for i, label in enumerate(labels):
                    data = dict()
                    data['image_id'] = int(frame_num)

                    x1 = bboxes[i][0]
                    y1 = bboxes[i][1]
                    x2 = bboxes[i][2]
                    y2 = bboxes[i][3]
                    coco_bbox = [x1, y1, x2-x1, y2-y1]
                    data['bbox'] = list(int(np.round(x)) for x in coco_bbox)
                    data['score'] = float(scores[i])
                    if data['score'] <= 0.7:
                        continue
                    data['category_id'] = categories[label]["id"]
                    if isinstance(masks[i]['counts'], bytes):
                        masks[i]['counts'] = masks[i]['counts'].decode()
                    data['segmentation'] = masks[i]
                    data['id'] = c + 1
                    c = c + 1
                    data["iscrowd"] = 0
                    coco_annotations.append(data)

                coco_image = {
                    "id": int(frame_num),
                    "width": 1920,
                    "height": 1080,
                    "file_name": framename + '.jpg',
                }

                coco_images.append(coco_image)
            else:
                print(f"Missing keys in result: {result.keys()}")




        return coco_images, coco_annotations

```markdown
# Test to Create a Single COCO File

In this section, we will test the creation of a single COCO file using the provided annotations and images. The `converter` function will be used to convert the annotations into COCO format, and the resulting data will be saved into a JSON file.

The following variables are used:
- `images`: Path to the directory containing the images.
- `annotations`: Path to the directory containing the annotation files.
- `output_file`: Path to the output JSON file where the COCO formatted data will be saved.

The process involves:
1. Converting the annotations to COCO format using the `converter` function.
2. Creating a dictionary with the necessary COCO information.
3. Saving the COCO formatted data into the specified output JSON file.
```
```

In [4]:
images = "../output/video_frames/417/VID003D" #images
annotations = "../data/ground_truth/417/VID003D/preds" #preds which is ground_truth
output_file = "../output/coco/417_VID003D.json" #output file
#'./vid511B_mask2former_swintransformerfixed_coco_score0.7.json' mask2former ground truth swin test random sampling

import json
import datetime
import os

coco_images, coco_annotations = converter(images, annotations)

info = {
    "date_created": str(datetime.datetime.now()),
    "description": "Automatically generated COCO json file for Detectron2.",
}
coco_dict = {"info": info, "images": coco_images, "categories": categories, "licenses": None}
if len(coco_annotations) > 0:

    coco_dict["annotations"] = coco_annotations

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

with open(output_file, 'w') as train_file:
    json.dump(coco_dict, train_file)


```markdown
# Create COCO Files for All Images in a Folder

The code in the following cell processes all image files in the specified `base_folder` and its subdirectories. For each folder containing image files, it converts the annotations to COCO format and saves the resulting data into a JSON file.

The key variables used in this process are:
- `base_folder`: Path to the base directory containing the video frames.
- `categories`: List of category definitions used in the COCO format.
- `converter`: Function that converts annotations to COCO format.
- `create_coco_for_all_files`: Function that processes all image files in the `base_folder` and creates COCO JSON files.

The process involves:
1. Iterating through all subdirectories in the `base_folder`.
2. For each subdirectory containing image files, constructing the paths for images, annotations, and the output JSON file.
3. Converting the annotations to COCO format using the `converter` function.
4. Creating a dictionary with the necessary COCO information.
5. Saving the COCO formatted data into the specified output JSON file.
```

In [6]:
import os
import json
import datetime

def create_coco_for_all_files(base_folder):
    for root, dirs, files in os.walk(base_folder):
        if any(file.endswith(('.png', '.jpg', '.jpeg')) for file in files):
            print(f"Folder: {root}")
            # get the last folder name and the parent folder name
            parent_folder = os.path.basename(os.path.dirname(root))
            folder_name = os.path.basename(root)

            print(f"Parent Folder: {parent_folder}")
            print(f"Folder Name: {folder_name}")

            images = f"../output/video_frames/{parent_folder}/{folder_name}" #images
            annotations = f"../data/ground_truth/{parent_folder}/{folder_name}/preds" #preds which is ground_truth
            output_file = f"../output/coco/{parent_folder}/{folder_name}.json" #output file

            # Ensure the output directory exists
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            
            coco_images, coco_annotations = converter(images, annotations)

            info = {
                "date_created": str(datetime.datetime.now()),
                "description": "Automatically generated COCO json file for samplingframework.",
            }
            coco_dict = {"info": info, "images": coco_images, "categories": categories, "licenses": None}
            if len(coco_annotations) > 0:
                coco_dict["annotations"] = coco_annotations

            with open(output_file, 'w') as train_file:
                json.dump(coco_dict, train_file)

base_folder = '../output/video_frames'
create_coco_for_all_files(base_folder)

Folder: ../output/video_frames\408\VID002A
Parent Folder: 408
Folder Name: VID002A
Missing keys in result: dict_keys(['images', 'annotations', 'categories'])
Folder: ../output/video_frames\408\VID002B
Parent Folder: 408
Folder Name: VID002B
Folder: ../output/video_frames\408\VID002C
Parent Folder: 408
Folder Name: VID002C
Folder: ../output/video_frames\417\VID003A
Parent Folder: 417
Folder Name: VID003A
Folder: ../output/video_frames\417\VID003D
Parent Folder: 417
Folder Name: VID003D
Folder: ../output/video_frames\432\VID001B
Parent Folder: 432
Folder Name: VID001B
Folder: ../output/video_frames\432\VID001C
Parent Folder: 432
Folder Name: VID001C
Folder: ../output/video_frames\496\VID001F
Parent Folder: 496
Folder Name: VID001F
Folder: ../output/video_frames\PB\VID001A
Parent Folder: PB
Folder Name: VID001A
Folder: ../output/video_frames\PB\VID001B
Parent Folder: PB
Folder Name: VID001B
